# Return Order Agent

## Imports

In [ ]:
# Core libraries
import os
import json
import sqlite3
import pandas as pd

# OpenAI client
from openai import OpenAI


In [ ]:
import pandas as pd

## Tools

In [ ]:
class PDFRetrieverTool:
    def __init__(self, content):
        self.content = content

    def run(self, query: str):
        if "return" in query.lower() or "refund" in query.lower():
            return self.content
        return "No relevant policy found in the PDF."


class DatabaseQueryTool:
    def __init__(self, db):
        self.db = db

    def run(self, query: str):
        """Execute an SQL query safely and return results as dictionaries."""
        try:
            result = self.db.run(query)
            
            # If the result is a string (some SQLDatabase backends return strings)
            if isinstance(result, str):
                import io
                return pd.read_csv(io.StringIO(result)).to_dict(orient="records")

            # Otherwise, assume it's a DataFrame
            return result.to_dict(orient="records")
        
        except Exception as e:
            return {"error": f"Error executing query: {e}"}


## Agent

In [ ]:
class AutonomousReturnAgent:
    def __init__(self, llm_client, tools: dict):
        self.client = llm_client
        self.tools = tools
        self.memory = []

    def build_prompt(self, user_message):
        memory_text = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in self.memory])
        return f"""
You are an autonomous customer service AI for Polar, a retail brand.
You can use the following tools when needed:

Tools available:
- pdf_tool: retrieves text from policy PDF.
- db_tool: queries customer and order database using SQL.

Your task is to assist with questions about orders, returns, and refunds.
You decide which tool to call and what SQL or query to run.
Always explain your reasoning and results clearly.

Conversation so far:
{memory_text}

User: {user_message}
If you need to use a tool, respond in JSON with the format:
{{"action": "tool_name", "input": "your query"}}
Otherwise, reply with the final message to the user.
"""

    def process(self, user_message):
        # Append user message to memory
        self.memory.append({"role": "user", "content": user_message})

        # Build reasoning prompt
        prompt = self.build_prompt(user_message)

        # Step 1: Ask LLM what to do
        response = self.client.chat.completions.create(
            model="gpt-5",
            messages=[{"role": "system", "content": "You are a helpful AI agent."},
                      {"role": "user", "content": prompt}]
        )
        message = response.choices[0].message.content.strip()
        print(f"🧰 LLM reasoning message: {message}")

        # Step 2: If the message looks like a tool call
        try:
            parsed = json.loads(message)
            if "action" in parsed:
                tool_name = parsed["action"]
                tool_input = parsed["input"]
                print(f"🧰 Using tool: {tool_name} → {tool_input}")

                if tool_name not in self.tools:
                    result = f"Unknown tool: {tool_name}"
                else:
                    result = self.tools[tool_name].run(tool_input)

                self.memory.append({"role": "tool", "content": str(result)})
                print(f"🧰 Tool result: {result}")

                # Ask LLM to summarize tool result for the user
                follow_up = self.client.chat.completions.create(
                    model="gpt-5",
                    messages=[
                        {"role": "system", "content": "You are a customer service AI assistant."},
                        {"role": "user", "content": f"The tool returned this: {result}. Summarize or continue the answer to the user clearly."}
                    ]
                )
                final_message = follow_up.choices[0].message.content.strip()
                self.memory.append({"role": "assistant", "content": final_message})
                print(f"🧰 Final message:")
                return final_message
        except Exception:
            print(f"🧰 Exception")
            pass  # not JSON, continue as normal text

        # Step 3: Otherwise, it’s a normal reply
        self.memory.append({"role": "assistant", "content": message})
        print(f"🧰 Final message:")
        return message


## Chat

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../docs/polar-return-policy.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

pdf_text = ""
for doc in docs:
    pdf_text += doc.page_content

print(pdf_text)

POLÍTICA DE TROCAS E DEVOLUÇÃO 
A Loja Virtual POLAR deseja sua total satisfação ao realizar suas compras. Neste sentido, criamos uma Política 
de Troca e Devolução respeitando às suas necessidades e baseada no Código de Defesa do Consumidor  
(http://www.idec.org.br/consultas/codigo-de-defesa-do-consumidor). As presentes condições de compra e venda 
são as cláusulas de contratação por adesão que você, consumidor, assume ao efetuar uma compra na Loja 
Virtual POLAR . 
Antes de qualquer procedimento , a troca ou devolução de qualquer produto precisa ser informada ao nosso 
Atendimento ao Cliente pelo telefone (11) 4935-2354, das 9h ás 18h, ou por e-mail lojavirtual@polar.com. 
DICAS IMPORTANTES 
Só serão aceitos produtos: não lavados, não usados, sem odores, que não tenham sido alterados, intactos, com 
as etiquetas originais e com sua embalagem original. Todo produto devolvido deverá vir acompanhado da Nota 
Fiscal, com as seguintes informações preenchidas no verso: 
Nome completo; 
En

In [ ]:
from langchain_community.utilities import SQLDatabase

local_db_path = "../datasets/olist_ecommerce.db"
db = SQLDatabase.from_uri(f"sqlite:///{local_db_path}")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM customers LIMIT 5;")}')

Dialect: sqlite
Available tables: ['category_translation', 'customers', 'geolocation', 'order_items', 'order_payments', 'order_reviews', 'orders', 'products', 'sellers']
Sample output: [('06b8999e2fba1a1fbc88172c00ba8bc7', '861eff4711a542e4b93843c6dd7febb0', 14409, 'franca', 'SP'), ('18955e83d337fd6b2def6b18a428ac77', '290c77bc529b7ac935b93aa66c333dc3', 9790, 'sao bernardo do campo', 'SP'), ('4e7b3e00288586ebd08712fdd0374a03', '060e732b5b29e8181a18229c7b0b2b5e', 1151, 'sao paulo', 'SP'), ('b2b6027bc5c5109e529d4dc6358b12c3', '259dac757896d24d7702b9acbbff3f3c', 8775, 'mogi das cruzes', 'SP'), ('4f2d8ab171c80ec8364f7c12e35b23ad', '345ecd01c38d18a9036ed96c73b8d066', 13056, 'campinas', 'SP')]


In [ ]:
pdf_tool = PDFRetrieverTool(pdf_text)
db_tool = DatabaseQueryTool(db)
client = OpenAI()

agent = AutonomousReturnAgent(
    llm_client=client,
    tools={"pdf_tool": pdf_tool, "db_tool": db_tool}
)


In [ ]:
print(agent.process("Hi! I want to know how the return policy works."))

🧰 LLM reasoning message: {"action": "pdf_tool", "input": "return policy"}
🧰 Using tool: pdf_tool → return policy
🧰 Tool result: POLÍTICA DE TROCAS E DEVOLUÇÃO 
A Loja Virtual POLAR deseja sua total satisfação ao realizar suas compras. Neste sentido, criamos uma Política 
de Troca e Devolução respeitando às suas necessidades e baseada no Código de Defesa do Consumidor  
(http://www.idec.org.br/consultas/codigo-de-defesa-do-consumidor). As presentes condições de compra e venda 
são as cláusulas de contratação por adesão que você, consumidor, assume ao efetuar uma compra na Loja 
Virtual POLAR . 
Antes de qualquer procedimento , a troca ou devolução de qualquer produto precisa ser informada ao nosso 
Atendimento ao Cliente pelo telefone (11) 4935-2354, das 9h ás 18h, ou por e-mail lojavirtual@polar.com. 
DICAS IMPORTANTES 
Só serão aceitos produtos: não lavados, não usados, sem odores, que não tenham sido alterados, intactos, com 
as etiquetas originais e com sua embalagem original. Todo 

In [ ]:
print(agent.process("Can you check if order e481f51cbdc54678b7cc49136f2d6af7 is eligible for a return?"))


🧰 LLM reasoning message: {"action": "db_tool", "input": "WITH o AS (\n  SELECT\n    o.id AS order_id,\n    o.customer_id,\n    o.status,\n    o.created_at,\n    COALESCE(o.delivered_at, s.delivered_at, d.delivered_at) AS delivered_at,\n    o.cancelled_at\n  FROM orders o\n  LEFT JOIN shipments s ON s.order_id = o.id AND (s.status ILIKE 'delivered%' OR s.status = 'DELIVERED')\n  LEFT JOIN deliveries d ON d.order_id = o.id AND (d.status ILIKE 'delivered%' OR d.status = 'DELIVERED')\n  WHERE o.id = 'e481f51cbdc54678b7cc49136f2d6af7'\n),\nr AS (\n  SELECT\n    r.order_id,\n    MIN(r.requested_at) AS first_return_requested_at,\n    MAX(r.status) FILTER (WHERE r.status IN ('approved','completed','refunded')) AS any_return_status\n  FROM returns r\n  WHERE r.order_id = 'e481f51cbdc54678b7cc49136f2d6af7'\n  GROUP BY r.order_id\n)\nSELECT\n  o.order_id,\n  o.customer_id,\n  o.status AS order_status,\n  o.created_at,\n  o.delivered_at,\n  o.cancelled_at,\n  r.first_return_requested_at,\n  r.any_

In [ ]:
print(agent.process("What is customer id for order 6514b8ad8028c9f2cc2374ded245783f ?"))


🧰 LLM reasoning message: {"action": "db_tool", "input": "PRAGMA table_info(orders);"}
🧰 Using tool: db_tool → PRAGMA table_info(orders);
🧰 Tool result: []


# New Agent

## SQL Toolkit

In [ ]:
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase

In [ ]:
local_db_path = "../datasets/olist_ecommerce.db"

In [ ]:
db = SQLDatabase.from_uri(f"sqlite:///{local_db_path}")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
sql_toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B5321DD250>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B5321DD250>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B5321DD250>),
 QuerySQLCheckerTool(description='Use this tool to 

In [ ]:
from langchain_classic import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [ ]:
system_template = prompt_template.messages[0].prompt.template
print(system_template)


You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see 

In [ ]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [ ]:
print(system_message)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [ ]:
sql_tools = sql_toolkit.get_tools()

NameError: name 'sql_toolkit' is not defined

In [ ]:
from langchain.agents import create_agent

sql_agent = create_agent(llm, sql_tools, system_prompt=system_message)

In [ ]:
example_query = "What is customer id for order 6514b8ad8028c9f2cc2374ded245783f ?"

events = sql_agent.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is customer id for order 6514b8ad8028c9f2cc2374ded245783f ?
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_ZV3tIcfOid6hfMDUodPwx7lT)
 Call ID: call_ZV3tIcfOid6hfMDUodPwx7lT
  Args:
    query: SELECT customer_id FROM orders WHERE order_id = '6514b8ad8028c9f2cc2374ded245783f'
================================= Tool Message =================================
Name: sql_db_query

[('9bdf08b4b3b52b5526ff42d37d47f222',)]
================================== Ai Message ==================================

The customer id for the order with order_id '6514b8ad8028c9f2cc2374ded245783f' is '9bdf08b4b3b52b5526ff42d37d47f222'.


## PDF Tool

In [ ]:
RETURN_POLICY_PATH = "../docs/polar-return-policy.pdf"

In [ ]:
from langchain.tools import tool
from langchain_community.document_loaders import PyPDFLoader

@tool(response_format="content_and_artifact")
def load_pdf():
    """Load PDF information."""
    loader = PyPDFLoader("../docs/polar-return-policy.pdf")
    docs = loader.load()
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in docs
    )
    return serialized, docs

In [ ]:
from langchain.chat_models import init_chat_model

llm_model = init_chat_model("openai:gpt-5")

In [ ]:
from langchain.agents import create_agent


tools = [load_pdf]
prompt = (
    "You have access to a tool that retrieves a PDF file containing the return policy. "
    "Use the tool to help answer user queries."
)
pdf_agent = create_agent(llm_model, tools, system_prompt=prompt)

In [ ]:
query = ("Hi! I want to know how the return policy works.")

for event in pdf_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! I want to know how the return policy works.
================================== Ai Message ==================================
Tool Calls:
  load_pdf (call_rNT8dvZQdCJXvQ6t2c2eZCiF)
 Call ID: call_rNT8dvZQdCJXvQ6t2c2eZCiF
  Args:
================================= Tool Message =================================
Name: load_pdf

Source: {'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2016-11-21T09:53:24+02:00', 'author': 'Tiago Lisboa', 'moddate': '2016-11-21T09:53:24+02:00', 'source': '../docs/polar-return-policy.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
Content: POLÍTICA DE TROCAS E DEVOLUÇÃO 
A Loja Virtual POLAR deseja sua total satisfação ao realizar suas compras. Neste sentido, criamos uma Política 
de Troca e Devolução respeitando às suas necessidades e baseada no Código de Defesa do Consumidor  
(http://www.idec.org.br/consultas/codigo-de-defesa-d

## Agent

In [ ]:
from typing import Annotated, Sequence, TypedDict

In [ ]:
class AgentState(TypedDict):
    """The state of the agent. Contais messages and metadata of the conversation.
    Args:
        messages: The historical data of the messages.
    """

    messages: Annotated[Sequence[BaseMessage], add_messages]